In [1]:
!pip install transformers torch requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import requests
import json
import torch
from transformers import pipeline

# Set Rocket.Chat API credentials
ROCKET_CHAT_URL = "https://your-rocketchat-server.com"
USERNAME = "your-username"
PASSWORD = "your-password"

# Authenticate and get the token
def get_auth_token():
    login_url = f"{ROCKET_CHAT_URL}/api/v1/login"
    data = {"user": USERNAME, "password": PASSWORD}

    response = requests.post(login_url, data=data)
    if response.status_code == 200:
        auth_data = response.json()
        return auth_data['data']['authToken'], auth_data['data']['userId']
    else:
        print("Login failed:", response.text)
        return None, None

# Fetch recent messages from a Rocket.Chat channel
def get_recent_messages(channel_name, auth_token, user_id):
    headers = {
        "X-Auth-Token": auth_token,
        "X-User-Id": user_id,
        "Content-Type": "application/json"
    }
    channel_url = f"{ROCKET_CHAT_URL}/api/v1/channels.history?roomName={channel_name}&count=10"

    response = requests.get(channel_url, headers=headers)
    if response.status_code == 200:
        messages = response.json().get("messages", [])
        return [msg["msg"] for msg in messages]
    else:
        print("Failed to fetch messages:", response.text)
        return []


In [3]:
# Load pre-trained transformer model for urgency classification
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Function to detect urgency in messages
def detect_urgent_messages(messages):
    urgent_messages = []

    for msg in messages:
        result = classifier(msg)
        label = result[0]['label']

        if label == "POSITIVE":  # Assume positive sentiment means urgent
            urgent_messages.append(msg)

    return urgent_messages


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Authenticate and fetch messages
auth_token, user_id = get_auth_token()
if auth_token:
    messages = get_recent_messages("general", auth_token, user_id)

    # Detect urgent messages
    urgent_messages = detect_urgent_messages(messages)

    print("\n🚨 Urgent Messages Detected 🚨")
    for msg in urgent_messages:
        print(f"- {msg}")


ConnectionError: HTTPSConnectionPool(host='your-rocketchat-server.com', port=443): Max retries exceeded with url: /api/v1/login (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7913d9377b50>: Failed to resolve 'your-rocketchat-server.com' ([Errno -2] Name or service not known)"))

In [8]:


import torch
from transformers import pipeline

# Load a more suitable transformer model (multi-label classification works better)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Keywords that indicate urgency
URGENT_KEYWORDS = ["urgent", "hurry", "asap", "immediately", "important", "critical", "emergency", "resolve soon", "high priority"]

def detect_urgency(text):
    # First, check if keywords indicate urgency
    if any(word in text.lower() for word in URGENT_KEYWORDS):
        return "🚨 It looks urgent!"

    # Use NLP model to classify urgency
    labels = ["urgent", "not urgent"]
    result = classifier(text, labels)

    # Check if "urgent" has a high confidence score
    if result["labels"][0] == "urgent" and result["scores"][0] > 0.7:
        return "🚨 It looks urgent!"
    else:
        return "✅ No urgency detected."

# Example: Input a message
message = input("Enter a message: ")
print(detect_urgency(message))


model.safetensors:  16%|#6        | 262M/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Enter a message: its an hurry , need to update soon
🚨 It looks urgent!
